<a href="https://colab.research.google.com/github/zyberg2091/Recognized-named-entities-in-twitter/blob/master/Recognize_Named_entities_in_twitter_with_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:
    ! wget https://raw.githubusercontent.com/hse-aml/natural-language-processing/master/setup_google_colab.py -O setup_google_colab.py
    import setup_google_colab
    setup_google_colab.setup_week2()

import sys
sys.path.append("..")
from common.download_utils import download_week2_resources

download_week2_resources()

--2020-08-02 12:02:38--  https://raw.githubusercontent.com/hse-aml/natural-language-processing/master/setup_google_colab.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1939 (1.9K) [text/plain]
Saving to: ‘setup_google_colab.py’

setup_google_colab. 100%[===================>]   1.89K  --.-KB/s    in 0s      

2020-08-02 12:02:39 (34.9 MB/s) - ‘setup_google_colab.py’ saved [1939/1939]



In [ ]:
import numpy as np

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras.layers import Input,Dense,SpatialDropout1D,Embedding,Bidirectional,LSTM,TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical

In [ ]:
def read_data(file_path):
    tokens = []
    tags = []
    
    tweet_tokens = []
    tweet_tags = []
    for line in open(file_path, encoding='utf-8'):
        line = line.strip()
        if not line:
            if tweet_tokens:
                tokens.append(tweet_tokens)
                tags.append(tweet_tags)
            tweet_tokens = []
            tweet_tags = []
        else:
            token, tag = line.split()
            if token.lower().startswith("https://"):
              token='<URL>'
            if token.lower().startswith("@"):
              token='<USR>'
          
            
            tweet_tokens.append(token)
            tweet_tags.append(tag)
            
    return tokens, tags

In [ ]:
train_tokens, train_tags = read_data('data/train.txt')
validation_tokens, validation_tags = read_data('data/validation.txt')
test_tokens, test_tags = read_data('data/test.txt')

In [ ]:
train_tokens[:2]

[['RT',
  '<USR>',
  ':',
  'Online',
  'ticket',
  'sales',
  'for',
  'Ghostland',
  'Observatory',
  'extended',
  'until',
  '6',
  'PM',
  'EST',
  'due',
  'to',
  'high',
  'demand',
  '.',
  'Get',
  'them',
  'before',
  'they',
  'sell',
  'out',
  '...'],
 ['Apple',
  'MacBook',
  'Pro',
  'A1278',
  '13.3',
  '"',
  'Laptop',
  '-',
  'MD101LL/A',
  '(',
  'June',
  ',',
  '2012',
  ')',
  '-',
  'Full',
  'read',
  'by',
  'eBay',
  'http://t.co/2zgQ99nmuf',
  'http://t.co/eQmogqqABK']]

In [ ]:
for i in range(3):
    for token, tag in zip(train_tokens[i], train_tags[i]):
        print('%s\t%s' % (token, tag))
    print()

RT	O
<USR>	O
:	O
Online	O
ticket	O
sales	O
for	O
Ghostland	B-musicartist
Observatory	I-musicartist
extended	O
until	O
6	O
PM	O
EST	O
due	O
to	O
high	O
demand	O
.	O
Get	O
them	O
before	O
they	O
sell	O
out	O
...	O

Apple	B-product
MacBook	I-product
Pro	I-product
A1278	I-product
13.3	I-product
"	I-product
Laptop	I-product
-	I-product
MD101LL/A	I-product
(	O
June	O
,	O
2012	O
)	O
-	O
Full	O
read	O
by	O
eBay	B-company
http://t.co/2zgQ99nmuf	O
http://t.co/eQmogqqABK	O

Happy	O
Birthday	O
<USR>	O
!	O
May	O
Allah	B-person
s.w.t	O
bless	O
you	O
with	O
goodness	O
and	O
happiness	O
.	O



In [ ]:
from collections import defaultdict

In [ ]:
def build_dict(tokens_or_tags,i):
    """
        tokens_or_tags: a list of lists of tokens or tags
        special_tokens: some special tokens
    """
    # Create a dictionary with default value 0
    tok2idx = defaultdict(lambda: 0)
    idx2tok = []
    
    # Create mappings from tokens (or tags) to indices and vice versa.
    # At first, add special tokens (or tags) to the dictionaries.
    # The first special token must have index 0.
    
    # Mapping tok2idx should contain each token or tag only once. 
    # To do so, you should:
    # 1. extract unique tokens/tags from the tokens_or_tags variable, which is not
    #    occur in special_tokens (because they could have non-empty intersection)
    # 2. index them (for example, you can add them into the list idx2tok
    # 3. for each token/tag save the index into tok2idx).

    vocab=set([t for ts in tokens_or_tags for t in ts])


    for key,value in enumerate(vocab):
      if i==0:
        tok2idx[value]=key+2
        idx2tok.append(value)
      if i==1:
        tok2idx[value]=key+1
        idx2tok.append(value)

  
    return tok2idx, idx2tok

In [ ]:

# Create dictionaries 
token2idx, idx2token = build_dict(train_tokens,0)
tag2idx, idx2tag = build_dict(train_tags,1)

In [ ]:
token2idx['<PAD>']=0
token2idx['<UNK>']=1
tag2idx['0']=0
idx2token.append('<UNK>')
idx2token.append('<PAD>')
idx2tag.append('0')

In [ ]:
def words2idxs(tokens_list):
    return [token2idx[word] for word in tokens_list]

def tags2idxs(tags_list):
    return [tag2idx[tag] for tag in tags_list]

def idxs2words(idxs):
    return [idx2token[idx] for idx in idxs]

def idxs2tags(idxs):
    return [idx2tag[idx] for idx in idxs]

In [ ]:
count=[]
for i in range(len(train_tokens)):
  count.append(len(train_tokens[i]))
length=max(count)

In [ ]:
#Data prep

x_train=[words2idxs(token) for token in train_tokens]
x_train_pad=pad_sequences(x_train,maxlen=length,value=token2idx['<PAD>'],padding="post")


y_train=[tags2idxs(tags) for tags in train_tags]
y_train_pad=pad_sequences(y_train,maxlen=length,value=tag2idx['0'],padding="post")

y_train_data=[to_categorical(y,num_classes=len(idx2tag)) for y in y_train_pad]

In [ ]:
s=[[1,2,3,4],[23,5,6,7]]
pad_sequences(s,maxlen=10,value=0)


array([[ 0,  0,  0,  0,  0,  0,  1,  2,  3,  4],
       [ 0,  0,  0,  0,  0,  0, 23,  5,  6,  7]], dtype=int32)

In [ ]:
x_val=[words2idxs(token) for token in validation_tokens]
x_val_pad=pad_sequences(x_val,maxlen=length,value=token2idx['<PAD>'],padding="post")

y_val=[tags2idxs(tags) for tags in validation_tags]
y_val_pad=pad_sequences(y_val,maxlen=length,value=tag2idx['0'],padding="post")

y_val_data=[to_categorical(y,num_classes=len(idx2tag)) for y in y_val_pad]

In [ ]:
y_train_data=np.array(y_train_data)

In [ ]:
x_train_pad.shape,y_train_data.shape

((5795, 41), (5795, 41, 22))

In [ ]:
#model architecture

i=Input(shape=(41,))
x=Embedding(len(idx2token)+1,64)(i)
x=SpatialDropout1D(0.1)(x)
x=Bidirectional(LSTM(64,return_sequences=True,recurrent_dropout=0.1))(x)
x=TimeDistributed(Dense(len(idx2tag),activation='softmax'))(x)


model=Model(i,x)
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

model.summary()


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 41)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 41, 64)            1344832   
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 41, 64)            0         
_________________________________________________________________
bidirectional (Bidirectional (None, 41, 128)           66048     
_________________________________________________________________
time_distributed (TimeDistri (None, 41, 22)            2838      
Total params: 1,413,718
Trainable params: 1,413,718
Non-trainable params: 0
_________________________________________________________________


In [ ]:
idx2tokens={value:key for key,value in token2idx.items()}
idx2tags={value:key for key,value in tag2idx.items()}
idx2tags

{0: '0',
 1: 'I-sportsteam',
 2: 'B-person',
 3: 'O',
 4: 'B-geo-loc',
 5: 'B-musicartist',
 6: 'B-other',
 7: 'I-person',
 8: 'I-geo-loc',
 9: 'B-movie',
 10: 'B-sportsteam',
 11: 'B-facility',
 12: 'I-facility',
 13: 'I-tvshow',
 14: 'B-product',
 15: 'I-product',
 16: 'B-tvshow',
 17: 'I-musicartist',
 18: 'I-movie',
 19: 'B-company',
 20: 'I-company',
 21: 'I-other'}

In [ ]:
r=model.fit(x_train_pad,y_train_data,validation_data=(x_val_pad,np.array(y_val_data)),epochs=10,verbose=1)

Epoch 1/10
182/182 [==============================] - 46s 252ms/step - loss: 0.5497 - accuracy: 0.8831 - val_loss: 0.4378 - val_accuracy: 0.9029
Epoch 2/10
182/182 [==============================] - 45s 249ms/step - loss: 0.1956 - accuracy: 0.9677 - val_loss: 0.4265 - val_accuracy: 0.9071
Epoch 3/10
182/182 [==============================] - 46s 254ms/step - loss: 0.1504 - accuracy: 0.9680 - val_loss: 0.4229 - val_accuracy: 0.9069
Epoch 4/10
182/182 [==============================] - 46s 251ms/step - loss: 0.1140 - accuracy: 0.9701 - val_loss: 0.4281 - val_accuracy: 0.9123
Epoch 5/10
182/182 [==============================] - 45s 249ms/step - loss: 0.0948 - accuracy: 0.9735 - val_loss: 0.4318 - val_accuracy: 0.9177
Epoch 6/10
182/182 [==============================] - 46s 252ms/step - loss: 0.0816 - accuracy: 0.9775 - val_loss: 0.4518 - val_accuracy: 0.9200
Epoch 7/10
182/182 [==============================] - 46s 251ms/step - loss: 0.0695 - accuracy: 0.9811 - val_loss: 0.4737 - val_ac

In [ ]:
x_val_pad

(724, 41)

In [ ]:
plt.plot(r.history['val_loss'],label='loss')
plt.plot(r.history['val_accuracy'],label='accuracy')
plt.legend()

In [ ]:
#predictions

p=model.predict(x_val_pad[1])

In [ ]:
y_true=np.argmax(p,axis=-1)

In [ ]:
y_true=y_true.reshape(1,41)[0]

In [ ]:
idx2tags

In [ ]:
for i,j in zip(x_val_pad[1],y_true):
  if j!=0:
    print("word : {} \t tag : {}".format(idx2tokens[i],idx2tags[j]))

In [ ]:
x_val_pad[1],y_true

In [ ]:
model.evaluate(x_val_pad,np.array(y_val_data))